In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

'2.9.2'

In [ ]:
# pip install tensorflow==2.9.1

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path 
from collections import Counter

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

model_loaded = keras.models.load_model('/content/final_model.h5', compile=False)
img_width = 100
img_height = 40
# Mapping integers back to original characters
num_to_char_1 = layers.experimental.preprocessing.StringLookup(
    vocabulary=['I',
 'N',
 'Q',
 'Y',
 'T',
 'F',
 'Z',
 'D',
 'X',
 'E',
 'P',
 '8',
 '2',
 'S',
 'L',
 'H',
 'M',
 'A',
 'V',
 'W',
 '5',
 '6',
 'C',
 'B',
 'R',
 '1',
 '7',
 '3',
 '9',
 'G',
 'U',
 'J',
 '4',
 'K'], mask_token=None, invert=True, num_oov_indices=0,oov_token=''
)


def encode_single_sample(img_path):
    # 1. Read image
    img = tf.io.read_file(img_path)
    # 2. Decode and convert to grayscale
    img = tf.io.decode_png(img, channels=1)
    # 3. Convert to float32 in [0, 1] range
    img = tf.image.convert_image_dtype(img, tf.float32)
    # 4. Resize to the desired size
    img = tf.image.resize(img, [img_height, img_width])
    # 5. Transpose the image because we want the time
    # dimension to correspond to the width of the image.
    img = tf.transpose(img, perm=[1, 0, 2])
    img = tf.expand_dims(img, axis=0)
    # 6. Map the characters in label to numbers
    preds = model_loaded.predict(img)
    pred_texts = decode_batch_predictions(preds)
    return pred_texts

# A utility function to decode the output of the network
def decode_batch_predictions(pred):
    input_len = np.ones(pred.shape[0]) * pred.shape[1]
    print(input_len)

    # Use greedy search. For complex tasks, you can use beam search
    results = keras.backend.ctc_decode(pred, input_length=input_len, greedy=True)[0][0][:, : 4]
    # Iterate over the results and get back the text
    output_text = []
    for res in results:
        res = tf.strings.reduce_join(num_to_char_1(res)).numpy().decode("utf-8")
        output_text.append(res)
    return output_text


encode_single_sample('/content/4L1T.png')

1/1 [==============================] - 2s 2s/step
[25.]


['4LT']